In [1]:
from pypekit import Task
import pandas as pd


class IrisLoader(Task):
    input_types = ["source"]
    output_types = ["raw"]

    def run(self, _):
        from sklearn.datasets import load_iris
        iris = load_iris()
        iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
        iris_df['target'] = iris.target
        return iris_df


class TrainTestSplitter(Task):
    input_types = ["raw"]
    output_types = ["split"]

    def run(self, df):
        from sklearn.model_selection import KFold
        kf = KFold(n_splits=self.run_config['n_splits'], shuffle=True, random_state=self.run_config['random_state'])
        split_index = self.run_config['split_index']
        train_indices, test_indices = list(kf.split(df))[split_index]

        train_df = df.iloc[train_indices].reset_index(drop=True)
        test_df = df.iloc[test_indices].reset_index(drop=True)

        train_df['train'] = 1
        test_df['train'] = 0
        df = pd.concat([train_df, test_df], ignore_index=True)
        return df


class Scaler(Task):
    input_types = ["split"]
    output_types = ["processed"]

    def run(self, df):
        X = df.drop(columns=['target', 'train'])
        X_train = X[df['train'] == 1]

        scaler = self.get_scaler()
        scaler.fit(X_train)

        X_scaled = scaler.transform(X)
        scaled_df = pd.DataFrame(data=X_scaled, columns=X.columns)
        scaled_df['target'] = df['target']
        scaled_df['train'] = df['train']

        return scaled_df

    def get_scaler(self):
        raise NotImplementedError("Subclasses should implement this method.")


class MinMaxScaler(Scaler):
    def get_scaler(self):
        from sklearn.preprocessing import MinMaxScaler
        return MinMaxScaler()


class StandardScaler(Scaler):
    def get_scaler(self):
        from sklearn.preprocessing import StandardScaler
        return StandardScaler()


class PCA(Task):
    input_types = ["processed"]
    output_types = ["pca"]

    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def run(self, df):
        X = df.drop(columns=['target', 'train'])
        X_train = X[df['train'] == 1]

        from sklearn.decomposition import PCA
        pca = PCA(**self.kwargs)
        pca.fit(X_train)

        X_pca = pca.transform(X)
        pca_df = pd.DataFrame(data=X_pca, columns=[
                              f'PC{i+1}' for i in range(X_pca.shape[1])])
        pca_df['target'] = df['target']
        pca_df['train'] = df['train']

        return pca_df


class Classifier(Task):
    input_types = ["split", "processed", "pca"]
    output_types = ["predicted"]

    def run(self, df):
        X = df.drop(columns=['target', 'train'])
        y = df['target']
        X_train = X[df['train'] == 1]
        y_train = y[df['train'] == 1]

        classifier = self.get_classifier()
        classifier.fit(X_train, y_train)
        
        y_pred = classifier.predict(X)
        df['predicted'] = y_pred

        return df
    
    def get_scaler(self):
        raise NotImplementedError("Subclasses should implement this method.")


class LogisticRegression(Classifier):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def get_classifier(self):
        from sklearn.linear_model import LogisticRegression
        return LogisticRegression(**self.kwargs)


class RandomForestClassifier(Classifier):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def get_classifier(self):
        from sklearn.ensemble import RandomForestClassifier
        return RandomForestClassifier(**self.kwargs)


class SVC(Classifier):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def get_classifier(self):
        from sklearn.svm import SVC
        return SVC(**self.kwargs)


class Evaluator(Task):
    input_types = ["predicted"]
    output_types = ["sink"]

    def run(self, df):
        df_test = df[df['train'] == 0]
        return (df_test['target'] == df_test['predicted']).mean()

In [2]:
from pypekit import Repository, CachedExecutor

repository = Repository([
    IrisLoader,
    TrainTestSplitter,
    MinMaxScaler,
    StandardScaler,
    PCA,
    LogisticRegression,
    RandomForestClassifier,
    SVC,
    Evaluator
])

repository.build_tree()
print(repository.build_tree_string())

└── Root
    └── IrisLoader
        └── TrainTestSplitter
            ├── MinMaxScaler
            │   ├── PCA
            │   │   ├── LogisticRegression
            │   │   │   └── Evaluator
            │   │   ├── RandomForestClassifier
            │   │   │   └── Evaluator
            │   │   └── SVC
            │   │       └── Evaluator
            │   ├── LogisticRegression
            │   │   └── Evaluator
            │   ├── RandomForestClassifier
            │   │   └── Evaluator
            │   └── SVC
            │       └── Evaluator
            ├── StandardScaler
            │   ├── PCA
            │   │   ├── LogisticRegression
            │   │   │   └── Evaluator
            │   │   ├── RandomForestClassifier
            │   │   │   └── Evaluator
            │   │   └── SVC
            │   │       └── Evaluator
            │   ├── LogisticRegression
            │   │   └── Evaluator
            │   ├── RandomForestClassifier
            │   │   └── Evaluator
            

In [3]:
pipelines = repository.build_pipelines()
pipelines

[Pipeline(tasks=['IrisLoader', 'TrainTestSplitter', 'MinMaxScaler', 'PCA', 'LogisticRegression', 'Evaluator']),
 Pipeline(tasks=['IrisLoader', 'TrainTestSplitter', 'MinMaxScaler', 'PCA', 'RandomForestClassifier', 'Evaluator']),
 Pipeline(tasks=['IrisLoader', 'TrainTestSplitter', 'MinMaxScaler', 'PCA', 'SVC', 'Evaluator']),
 Pipeline(tasks=['IrisLoader', 'TrainTestSplitter', 'MinMaxScaler', 'LogisticRegression', 'Evaluator']),
 Pipeline(tasks=['IrisLoader', 'TrainTestSplitter', 'MinMaxScaler', 'RandomForestClassifier', 'Evaluator']),
 Pipeline(tasks=['IrisLoader', 'TrainTestSplitter', 'MinMaxScaler', 'SVC', 'Evaluator']),
 Pipeline(tasks=['IrisLoader', 'TrainTestSplitter', 'StandardScaler', 'PCA', 'LogisticRegression', 'Evaluator']),
 Pipeline(tasks=['IrisLoader', 'TrainTestSplitter', 'StandardScaler', 'PCA', 'RandomForestClassifier', 'Evaluator']),
 Pipeline(tasks=['IrisLoader', 'TrainTestSplitter', 'StandardScaler', 'PCA', 'SVC', 'Evaluator']),
 Pipeline(tasks=['IrisLoader', 'TrainTes

In [4]:
executor = CachedExecutor(pipelines, verbose=True)

n_splits = 5
results_dfs = []
for i in range(n_splits):
    results = executor.run(run_config={"n_splits": 5, "split_index": i, "random_state": 42})
    results = pd.DataFrame(results).transpose()
    results.reset_index(inplace=True, names="pipeline")
    results_dfs.append(results)

Pipeline 1/15 completed. Runtime: 0.44s.
Pipeline 2/15 completed. Runtime: 0.53s.
Pipeline 3/15 completed. Runtime: 0.43s.
Pipeline 4/15 completed. Runtime: 0.41s.
Pipeline 5/15 completed. Runtime: 0.48s.
Pipeline 6/15 completed. Runtime: 0.41s.
Pipeline 7/15 completed. Runtime: 0.41s.
Pipeline 8/15 completed. Runtime: 0.52s.
Pipeline 9/15 completed. Runtime: 0.41s.
Pipeline 10/15 completed. Runtime: 0.41s.
Pipeline 11/15 completed. Runtime: 0.49s.
Pipeline 12/15 completed. Runtime: 0.40s.
Pipeline 13/15 completed. Runtime: 0.41s.
Pipeline 14/15 completed. Runtime: 0.50s.
Pipeline 15/15 completed. Runtime: 0.40s.
Pipeline 1/15 completed. Runtime: 0.01s.
Pipeline 2/15 completed. Runtime: 0.10s.
Pipeline 3/15 completed. Runtime: 0.01s.
Pipeline 4/15 completed. Runtime: 0.01s.
Pipeline 5/15 completed. Runtime: 0.09s.
Pipeline 6/15 completed. Runtime: 0.01s.
Pipeline 7/15 completed. Runtime: 0.01s.
Pipeline 8/15 completed. Runtime: 0.09s.
Pipeline 9/15 completed. Runtime: 0.01s.
Pipeline 1

In [5]:
result_df = pd.concat(results_dfs)
accuracies_df = result_df.groupby("pipeline").agg(
    tasks=("tasks", "first"),
    mean_runtime=("runtime", "mean"),
    mean_accuracy=("output", "mean"),
)
accuracies_df.sort_values("mean_accuracy", ascending=False, inplace=True)
accuracies_df

,tasks,mean_runtime,mean_accuracy
pipeline,,,
be05d37704c041fcb2389913f753e135,"[IrisLoader, TrainTestSplitter, LogisticRegres...",0.090743,0.973333
2814cd7bd66f42b696de94adbb6db84f,"[IrisLoader, TrainTestSplitter, SVC, Evaluator]",0.084459,0.966667
6970e701fd85433c88c314bddeb58108,"[IrisLoader, TrainTestSplitter, RandomForestCl...",0.16156,0.966667
62c7456e49cb4cb29d0dc2807ae51ed5,"[IrisLoader, TrainTestSplitter, MinMaxScaler, ...",0.171667,0.96
99629db9eb8e4ce1aa1959fc7fa3dfec,"[IrisLoader, TrainTestSplitter, MinMaxScaler, ...",0.085993,0.96
a1b72f7ff33a48e48cb62d8bf8cacca8,"[IrisLoader, TrainTestSplitter, MinMaxScaler, ...",0.15465,0.96
e0c87bd42a7c4f39875682770b96da5c,"[IrisLoader, TrainTestSplitter, MinMaxScaler, ...",0.091546,0.96
27ddb9fe99d1433e894b669b780d93bc,"[IrisLoader, TrainTestSplitter, StandardScaler...",0.160816,0.953333
76cd064a0c07476f837919ad6a7d4dc9,"[IrisLoader, TrainTestSplitter, StandardScaler...",0.085872,0.953333
